In [ ]:
### Project 2 SQLAlchemy Assignment Using Yeast_RNA-Seq Reads sqlite database 

In [ ]:
#!/usr/bin/env python
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String
from sqlalchemy import sql, select, join, desc
import pandas as pd

import sqlite3

#Importing the database and establishing a connection
yeast_engine = create_engine('sqlite:////blue/bsc4452/admontalvo/yeast.sqlite')
conn=yeast_engine.connect()

#creating a dataframe with pandas combining
#gene_id,location_id,function_id, and bioprocess_id to create a unique gene ID
#Odd primary key 
#Join statement linking all the tables together
df_yeast=pd.read_sql_query('''SELECT Expression.transcripts, Genes.name, Description.meaning,Molecular.function,Treatment.condition,locations.location,Bioprocess.process
                  FROM Expression JOIN Genes JOIN Description JOIN Molecular JOIN Treatment JOIN locations JOIN Bioprocess
                ON Expression.gene_id=Genes.id AND Expression.meaning_id=Description.id AND Expression.treatment_id=Treatment.id
                AND Expression.function_id=Molecular.id AND Expression.location_id=locations.id AND Expression.bioprocess_id=Bioprocess.id''', conn)


df_yeast.head(10) #Prints the first 10 rows

In [ ]:
#obtaining the names of all the tables with the engine connection
yeast_engine.table_names()

In [ ]:
#obtaining table descriptions from database, similiar to .schema command function with SQLite3 in the terminal
#error potentially is present from sqlite_sequence table and possibly no values are present there
from sqlalchemy import create_engine, inspect

#inspector valueto look at the table
inspector = inspect(yeast_engine)
table_names = inspector.get_table_names()
for table_name in table_names:
    print(f"Table:{table_name}")
    column_items = inspector.get_columns(table_name)
    print('\t'.join(n for n in column_items[0]))
    for c in column_items:
        assert len(c) == len(column_items[0])
        print('\t'.join(str(c[n]) for n in c))

In [ ]:
#Looking at the presentation of the cellular compartments from the locations table
yeast_locations = yeast_engine.execute("SELECT * FROM locations").fetchall()
print(yeast_locations)

In [ ]:
#visualing how transcripts from the Expression table is presented in the database by looking at one results with .fetchone()
#The output appear to represent the gene_id, transcripts , treatment_id, location_id, 
#meaning_id, function_id and bioprocess_id from the Expression table. We are interested in the transcripts
yeast_transcripts = yeast_engine.execute("SELECT * FROM Expression").fetchall() #.fetchall() can print out all the outputs 
print(yeast_transcripts)

In [ ]:
#function to determine the total number of counts RNA -seq reads from each
#cell compartment from the locations coloumn
def filter_transcripts(my_df_transcripts, transcripts): #variables in parenthesis to define the dataframe and location
    filtered_df_transcripts = my_df_transcripts['transcripts'] == transcripts
    return filtered_df_transcripts, sum(filtered_df_transcripts) #returns filtered transcript reads and total counts of the values

In [ ]:
#If statement for transcript reads > 5
yeast_read_counts = {}
for transcripts in yeast_transcripts:
    print(transcripts)
    yeast_reads, yeast_read_total = filter_transcripts(df_yeast, df_yeast['transcripts'])
    yeast_read_counts[transcripts[2]] = yeast_read_total
    if df_yeast['transcripts'] > 5:
        print(yeast_read_total)

In [ ]:
#function to determine the total number of counts RNA -seq reads from each
#cell compartment from the locations coloumn
def filter_location(my_df, location): #variables in parenthesis to define the dataframe and location
    filtered_df = my_df['location'] == location
    return filtered_df, sum(filtered_df) #returns cell compartments and total counts of the values

In [ ]:
#Obtaining the total number RNA-seq counts per cell compartment using a dictionary
location_counts = {}

# For loop to go line by line in the location column and 
for location in yeast_locations:
    print(location[1]) #location is the desired column
    location_mem,location_total = filter_location(df_yeast, location[1]) #two variables placed before the equal sign 
    #print(location_mem, location_total)
    location_counts[location[1]] = location_total
print(location_counts) 

In [ ]:
# Pie chart plotting the RNA - seq reads fromt he sql  Vs other values using matplotlib
import numpy as np
import matplotlib.pyplot as plt

#Customizing the pie long, medium and short chart and playing around with the functions
labels = 'Membrane', 'Mitochondrial', 'Nucleus', 'Cytoplasm', 'Cytosol', 'Endoplasm', 'Extracellular'
sizes = [10741, 3565, 14168, 2829, 1058, 1196, 115]
 
colors = ( "orange", "cyan", "brown", "green", "red", "blue", "yellow") 
fig1, ax1 = plt.subplots()
#Can change the value to make the slice of the pie farther or closer 
#in this case the extracellular, endoplasm, cytosol, and cytoplasm sections are extended 
explode = (0, 0, 0, 0.1, 0.25, 0.4, 0.5) 

#Customizing the pie chart explode makes a slice pop out startangle rotates it
ax1.pie(sizes, colors = colors, explode=explode, labels=labels,autopct='%1.1f%%', shadow=True, startangle=90)
patches, texts, auto = ax1.pie(sizes, colors=colors, shadow=True, startangle=90,explode=explode, autopct='%1.1f%%' )

plt.title('Number of RNA-seq Reads per Cellular comaprtment from the yeast.sqlite database') 
#plt.legend(patches, labels, loc="best") #legend is poorly placed but I mostly wanted to work with the code
#the legend can be moved with upper left, lower right, upper right, designitions but all of them look worse
plt.show()

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
 
  
#creating the dataset

data = {'Membrane':10741, 'Mitochondrial':3565, 'Nucleus':14168, 'Cytoplasm':2829, 'Cytosol':1058, 'Endoplasm':1196, 
        'Extracellular':115}
cell_compartment = list(data.keys()) #x axis displayed as a list
rna_seq_reads = list(data.values()) #y axis dispalyed as a list
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(cell_compartment, rna_seq_reads, color ='maroon',
        width = 0.4)

#Creating the title, 
plt.xlabel("Cellular Compartment")
plt.ylabel("Quanitity of RNA-seq Reads")
plt.title("Number of RNA-Sequancing Reads Separated Cellular comaprtment from the yeast.sqlite database")
plt.show()


In [ ]:
#function to determine the total number of counts RNA -seq reads from each
#cell compartment from the locations coloumn
def filter_transcript(my_df, transcript): #variables in parenthesis to define the dataframe and location
    filtered_transcripts_df = my_df['transcripts'] == transcripts
    return filtered_transcripts_df, sum(filtered_transcripts_df) #returns cell compartments and total counts of the values

In [ ]:
#Obtaining the total number RNA-seq counts per cell compartment using a dictionary
transcript_counts = {}

# For loop to go line by line in the location column and transcripts column and obtian the number of reads with transcript values > 5
for transcript in yeast_transcripts:
    if filtered_transcripts_df > 5:
        print(transcripts)
        transcripts_count,transcript_total = filter_transcript(df_yeast, transcripts)
        #print(location_mem, location_total)
        transcript_counts[transcripts] = transcript_total
    print(transcript_counts)

In [ ]:
# Filtering values within the location column in the yeast database that contain the term membrane
location_mem = df_yeast["location"] == "membrane"
print(location_mem)
sum(location_mem)

In [ ]:
#Sorting dataframe to obtain yeast strains in the different compartments with RNA-seq transcript values < 5 and FFNAAA growth treatment

df_yeast.sort_values("transcripts", inplace = True)
  
# Filtering transcripts < 5  rna seq reads from yeast database
transcripts_filter1 = df_yeast["transcripts"] < 5

# Filtering transcripts > 5  rna seq reads from yeast database
transcripts_filter2 = df_yeast["transcripts"] > 5

# Filtering values within the location column in the yeast database that contain the term membrane
location_mem = df_yeast["location"] == "membrane"

# Filtering values within the location column in the yeast database that contain the term "mitochond" which is short for mitochondria
location_mito = df_yeast["location"] == "mitochond"

# Filtering values within the location column in the yeast database that contain the term "nucle" which is short for nucleus
location_nucl = df_yeast["location"] == "nucle"

# Filtering values within the location column in the yeast database that contain the term "cytoplasm" 
location_cytoplasm = df_yeast["location"] == "cytoplasm"

# Filtering values within the location column in the yeast database that contain the term "cytosol" 
location_cytosol = df_yeast["location"] == "cytosol"

# Filtering values within the location column in the yeast database that contain the term "endoplasm" 
location_endoplasm = df_yeast["location"] == "endoplasm"

# Filtering values within the location column in the yeast database that contain the term "extracellular" 
location_extracellular = df_yeast["location"] == "extracellular"


#displaying the new dataframe with filtered data
df_yeast





In [ ]:
# Filtering transcripts < 5  rna seq reads from yeast database Testing this out
transcripts_filter1 = df_yeast["transcripts"] < 5
print(transcripts_filter1)